In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from HelperFunctions import *
from PtOccupancyFunctions import *

# Get Data

In [ ]:
def getData(directory):
    #     end_time = 24 * 3600  # 24 hours
    for sdir_path, sdir in [
        (sdir.path, sdir.name)
        for sdir in sorted(
            os.scandir(directory), key=lambda x: int(x.name.split("_")[-1])
        )
        if sdir.is_dir()
    ]:
        print(sdir)
        for root, subdirs, files in os.walk(sdir_path):
            for file in files:
                dirs = root.split("/")
                if file == "0.trips.csv.gz" and "car" in dirs:
                    dists_car = pd.read_csv(os.path.join(root, file), sep=";")[
                        "traveled_distance"
                    ]
                if file == "0.vehicleDistanceStats_drt.csv" and "bimodal" in dirs:
                    dists_drt = pd.read_csv(os.path.join(root, file), sep=";")[
                        "drivenDistance_m"
                    ]
                if file == "trip_success.csv.gz" and "bimodal" in dirs:
                    rejections = pd.read_csv(os.path.join(root, file), sep=";")
                if file == "0.CummulativePtDistance.txt" and "bimodal" in dirs:
                    dist_train = pd.read_csv(os.path.join(root, file)).values[0, 0]
                if (
                    file == "0.drt_occupancy_time_profiles_drt.txt"
                    and "bimodal" in dirs
                ):
                    df_drt_occ = pd.read_csv(os.path.join(root, file), sep="\t")
                    df_drt_occ["time"] = df_drt_occ["time"].apply(timestmphm2sec)
                    drt_occ, drt_deviation = getAverageOcc(
                        df_drt_occ.drop(columns="time"), exclude_empty_vehicles=True
                    )
                if file == "output_trips.csv.gz" and "bimodal" in dirs:
                    df_trips = pd.read_csv(os.path.join(root, file), sep=";").loc[
                        :,
                        [
                            "person",
                            "trav_time",
                            "wait_time",
                            "traveled_distance",
                            "modes",
                        ],
                    ]
                    df_trips["trav_time"] = df_trips["trav_time"].apply(timestmp2sec)
                    df_trips["wait_time"] = df_trips["wait_time"].apply(timestmp2sec)
                    df_trips.set_index("person", inplace=True)
                    df_trips = df_trips.groupby("person").agg(
                        {
                            "trav_time": np.sum,
                            "wait_time": np.sum,
                            "traveled_distance": np.sum,
                            "modes": combineModesSeriesStr,
                        }
                    )
                if file == "0.occupancyAnalysis.txt" and "bimodal" in dirs:
                    av_pt_occ, av_pt_occ_sq, n_pt = getPtOccupancies(
                        os.path.join(root, file), 600
                    )
                    t_av_pt_occ_av = getAverageTimeSeries(av_pt_occ)
                    sigma = np.sqrt(n_pt / (n_pt - 1)) * np.sqrt(
                        av_pt_occ_sq - av_pt_occ ** 2
                    )
                    t_av_pt_occ_sigma = getAverageTimeSeries(sigma)
                if file == "0.drt_trips_drt.csv" and "bimodal" in dirs:
                    drt_trip_dists = pd.read_csv(os.path.join(root, file), sep=";").loc[:,
                        "travelDistance_m"
                    ]

        yield int(sdir.split("_")[-1]), dists_car, dists_drt, dist_train, rejections, (
            drt_occ,
            drt_deviation,
        ), df_trips["wait_time"].mean(), (t_av_pt_occ_av, t_av_pt_occ_sigma), df_trips[
            "modes"
        ].str.contains(
            "pt"
        ), df_trips[
            "traveled_distance"
        ], drt_trip_dists

In [ ]:
path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/varyingL_lMaxDelta3/"
%matplotlib notebook

L_ls = np.empty(9)
carbon_indexs = np.empty(9)
cummulative_drt_car_bimodal = np.empty(9)
cummulative_train_bimodal = np.empty(9)
cummulative_car = np.empty(9)
av_drt_occs = np.empty(9)
av_waits = np.empty(9)
serviced_fracts = np.empty(9)
av_pt_occs = np.empty(9)
bi_or_unimodal = np.empty((9,2))
drt_pt_person_km = np.empty((9,2))

r0 = 3
# for x in getData(path):
#     fig,ax = plt.subplots()
#     x.plot(ax=ax)
#     continue
# plt.show()

for i, (
    L_l,
    dists_car,
    dists_drt,
    dist_train,
    rejections,
    (drt_occ, drt_occ_deviation),
    av_wait_time,
    (pt_occ, pt_occ_sigma),
    pt_in_trips,
    total_trip_dists,
    drt_trip_dists
) in enumerate(getData(path)):
    
    drt_pt_person_km[i,0] = drt_trip_dists.sum()
    drt_pt_person_km[i,1] = total_trip_dists.sum()-drt_pt_person_km[i,0]
    L_ls[i] = L_l
    serviced_fracts[i] = len(rejections[rejections["tripSuccess"]]) / len(rejections)
    print("Servability: ", serviced_fracts[i])
    dists_car_for_rejected = dists_car.loc[
        rejections[~rejections["tripSuccess"]]["personId"].to_numpy()
    ]
    cummulative_drt_car_bimodal[i] = dists_drt.sum() + dists_car_for_rejected.sum()
    print("Distance by drt: ", dists_drt.sum())
    cummulative_train_bimodal[i] = dist_train
    cummulative_car[i] = dists_car.sum()
    carbon_index = (
        cummulative_drt_car_bimodal[i] + r0 * cummulative_train_bimodal[i]
    ) / cummulative_car[i]
    carbon_indexs[i] = carbon_index
    print(L_l)
    print(carbon_index)
    av_drt_occs[i] = drt_occ
    av_waits[i] = av_wait_time
    av_pt_occs[i] = pt_occ
    vc_modes = pt_in_trips.value_counts()
    bi_or_unimodal[i,0] = vc_modes[False]
    bi_or_unimodal[i,1] = vc_modes[True]
    print("-----")

bi_or_unimodal = pd.DataFrame(bi_or_unimodal, index=L_ls, columns=["Unimodal","Bimodal"])
drt_pt_person_km = pd.DataFrame(drt_pt_person_km, index=L_ls, columns=["DRT","PT"])

---
# Visualization

In [ ]:
%matplotlib inline
save_figs = False

fig, ax = plt.subplots()
x = 1/L_ls
xlabel = r"$\ell/L$"

for r0 in [1,2,3,4,5]:
    carbon_inds = (cummulative_drt_car_bimodal+r0*cummulative_train_bimodal)/cummulative_car
    ax.plot(x, carbon_inds, 'o-', label="R0 = {}".format(r0))
#     print(np.argmin(carbon_inds))
    ax.set_ylabel(r"$C_I$")
    ax.set_xlabel(xlabel)
    ax.legend()

if save_figs:
    fig.savefig("CarbonIdxVaryingLl.png", dpi=200)

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

# bi_or_unimodal.set_index(round(1/bi_or_unimodal.reset_index()["index"]), inplace=True)
bi_or_unimodal.set_index(np.round(x, decimals=2), inplace=True)
bi_or_unimodal.plot.bar(ax = ax, xlabel=xlabel, ylabel="frequency")

if save_figs:
    fig.savefig("BiOrUnimodalBarPlot.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

# bi_or_unimodal.set_index(round(1/bi_or_unimodal.reset_index()["index"]), inplace=True)
drt_pt_person_km.set_index(np.round(x, decimals=2), inplace=True)
drt_pt_person_km.plot.bar(ax = ax, xlabel=xlabel, ylabel="meter travelled")

patches, labels = ax.get_legend_handles_labels()

ax.legend(patches, labels, frameon=False, bbox_to_anchor=(1.05, 1))

# save_figs = True
if save_figs:
    fig.savefig("PersonKmPerTravelMode.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x, serviced_fracts, 'o-')
ax.set_ylabel("f")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("ServicedFraction.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x, av_waits, 'o-')
ax.set_ylabel("Average waiting time (s)")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AverageWaitingTime.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x, av_drt_occs, 'o-')
ax.set_ylabel(r"$\langle b_\mathrm{drt}^*\rangle$")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AverageNonEmptyOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x, av_pt_occs, 'o-')
ax.set_ylabel(r"$\langle b_\mathrm{pt}\rangle$")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AveragePtOccupancy.png", dpi=200)
plt.show()

In [ ]:
print(wait2400/serviced_fracts)

In [ ]:
a = np.array([0.1, 0.2, .3, 0.4, 0.5])
print(1/a)

In [ ]:
print(10000/5500)